In [8]:
from bsf_env import init_spark, init_mariadb_engine,set_spark_verbosity
from bsf_dbutilities import DBUtils
from pyspark.sql import functions as F
import pandas as pd

try:
    spark.stop()
except NameError:
    # SparkSession doesn't exist
    pass

spark = init_spark("bsf_utilities", log_level="WARN", show_progress=False, enable_ui=True,process_option='manual')
engine = init_mariadb_engine()
ingest_ts = spark.sql("SELECT current_timestamp()").collect()[0][0]

[Spark] Started 'bsf_utilities' log_level=WARN (effective=WARN), progress=False


25/10/03 21:10:58 WARN SQLConf: The SQL config 'spark.sql.adaptive.shuffle.targetPostShuffleInputSize' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.adaptive.advisoryPartitionSizeInBytes' instead of it.
25/10/03 21:10:59 WARN SQLConf: The SQL config 'spark.sql.adaptive.shuffle.targetPostShuffleInputSize' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.adaptive.advisoryPartitionSizeInBytes' instead of it.
25/10/03 21:10:59 WARN SQLConf: The SQL config 'spark.sql.adaptive.shuffle.targetPostShuffleInputSize' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.adaptive.advisoryPartitionSizeInBytes' instead of it.


In [9]:
# ─── Setup Database Communications ──────────────────────────────────────
db = DBUtils(spark, ingest_ts)
db.spark_stats()

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 200)         # Expand width to avoid line breaks


from delta.tables import DeltaTable
import datetime, os
from pyspark.sql import functions as F

# List all databases
databases = [db.name for db in spark.catalog.listDatabases()]

for database in databases:
    print(f"\n=== Database: {database} ===\n")
    
    # Get all tables in this database
    tables = spark.catalog.listTables(database)
    if not tables:
        print("No tables found.")
        continue

    table_info = []
    
    for t in tables:
        full_table_name = f"{database}.{t.name}"
        row_count = spark.table(full_table_name).count()
        table_type = t.tableType
    
        # Default: no max date
        from pyspark.sql import functions as F
        import datetime
        import os
        
        max_date = None
        try:
            df_table = spark.table(full_table_name)
            
            if "LastLoadedDate" in df_table.columns:
                max_date = df_table.agg(F.max("LastLoadedDate")).collect()[0][0]
                field = "LastLoadedDate"
            elif "StockDate" in df_table.columns:
                max_date = df_table.agg(F.max("StockDate")).collect()[0][0]
                field = "StockDate"
            elif "ChangeDate" in df_table.columns:
                max_date = df_table.agg(F.max("ChangeDate")).collect()[0][0]
                field = "ChangeDate"
            else:
                try:
                    dt = DeltaTable.forName(spark, full_table_name)
                    history = dt.history(1)
                    max_date = history.select(F.max("timestamp")).collect()[0][0]
                    field = "timestamp"
                except:
                    max_date = None
        
            # --- Normalize to datetime ---
            if isinstance(max_date, datetime.date) and not isinstance(max_date, datetime.datetime):
                max_date = datetime.datetime.combine(max_date, datetime.time.min)
        
        except:
            # Non-Delta fallback
            table_location = (
                spark.sql(f"DESCRIBE FORMATTED {full_table_name}")
                     .filter("col_name='Location'")
                     .select("data_type")
                     .collect()[0][0]
            )
            max_date = datetime.datetime.fromtimestamp(os.path.getmtime(table_location))


    
        table_info.append((t.name, row_count, field, max_date, table_type))
    
    df_table_info = spark.createDataFrame(
        table_info,
        schema=["TableName", "RowCount","DateField", "MaxDate", "TableType"]
    )
    
    df_table_info.show(truncate=False)
    # 🚨 Safety stop — prevents accidental full execution
    raise RuntimeError("⚠️ This notebook is blocked. Do NOT run all cells without checking!")

    🔍 === Runtime Config Check ===
      📌 User                       : jupyter
      📌 Python Version             : 3.9.21
    🔍 === Spark Runtime Config Check ===
      📌 Name                       : bsf_utilities
      📌 Master                     : local[1]
      📌 Version                    : Not set
      📌 Max Cores                  : 1
      📌 Executor Instances         : 1
      📌 Executor Cores             : 1
      📌 Task Cpus                  : 1
      📌 Executor Memory            : 512m
      📌 Driver Memory              : 512m


25/10/03 21:11:51 WARN SQLConf: The SQL config 'spark.sql.adaptive.shuffle.targetPostShuffleInputSize' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.adaptive.advisoryPartitionSizeInBytes' instead of it.
25/10/03 21:11:54 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/10/03 21:11:54 WARN HiveConf: HiveConf of name hive.metastore.client.connect.timeout does not exist
25/10/03 21:11:54 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/10/03 21:12:03 WARN SQLConf: The SQL config 'spark.sql.adaptive.shuffle.targetPostShuffleInputSize' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.adaptive.advisoryPartitionSizeInBytes' instead of it.



=== Database: bsf ===



25/10/03 21:12:24 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
25/10/03 21:12:27 WARN SQLConf: The SQL config 'spark.sql.adaptive.shuffle.targetPostShuffleInputSize' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.adaptive.advisoryPartitionSizeInBytes' instead of it.


+-----------------------------+--------+----------+--------------------------+---------+
|TableName                    |RowCount|DateField |MaxDate                   |TableType|
+-----------------------------+--------+----------+--------------------------+---------+
|company                      |31051   |ChangeDate|2025-09-30 09:03:07.840831|MANAGED  |
|companyfundamental           |283118  |timestamp |2025-10-03 09:14:28.767   |MANAGED  |
|companystockhistory          |548957  |StockDate |2025-10-02 00:00:00       |MANAGED  |
|companystockhistory_watermark|6171    |timestamp |2025-10-03 09:12:58.554   |MANAGED  |
|final_candidates             |25      |StockDate |2025-10-02 00:00:00       |MANAGED  |
|final_candidates_enriched    |110     |StockDate |2025-10-02 00:00:00       |MANAGED  |
|history_signal_driver        |546731  |StockDate |2025-10-02 00:00:00       |MANAGED  |
|history_signals              |2733655 |StockDate |2025-10-02 00:00:00       |MANAGED  |
|history_signals_last

RuntimeError: ⚠️ This notebook is blocked. Do NOT run all cells without checking!

In [7]:
spark.stop()

In [ ]:
sdf = spark.table("bsf.company")
sdf.toPandas().describe()


In [22]:
from copy import deepcopy
from bsf_config import CONFIG

In [24]:
def load_settings(profile: str = "default" ):
    """Load settings, merging defaults with optional profile overrides.
       For `timeframe_map`, profile overrides REPLACE the whole dict
       instead of merging.
    """
    default_settings = deepcopy(CONFIG["default"])
    settings = deepcopy(default_settings)

    if profile and profile in CONFIG:
        overrides = CONFIG[profile]

        def merge(base, update, path=""):
            for k, v in update.items():
                current_path = f"{path}.{k}" if path else k

                # Special case: timeframe_map is replace, not merge
                if k == "timeframe_map":
                    old_value = base.get(k, "<not in default>")
                    #print(f"Override: {current_path}: default={old_value}, profile={v}")
                    base[k] = deepcopy(v)
                    continue

                if isinstance(v, dict) and k in base and isinstance(base[k], dict):
                    merge(base[k], v, current_path)
                else:
                    old_value = base.get(k, "<not in default>")
                    #print(f"Override: {current_path}: default={old_value}, profile={v}")
                    base[k] = deepcopy(v)

        merge(settings, overrides)

    return settings


In [28]:
import json
mbsetting = load_settings()

config_json = json.dumps(mbsetting)
print(config_json)

{"timeframe_map": {"Daily": 1, "Short": 3, "Swing": 5, "Long": 10}, "phases": {"phase1": {"topN": 50}, "phase2": {"topN": 25}, "phase3": {"topN": 5}}, "profiles": {"Short": {"ma": 2, "ret": 1, "vol": 3, "roc_thresh": 0.02, "slope_horizon": 1}, "Swing": {"ma": 5, "ret": 5, "vol": 5, "roc_thresh": 0.02, "slope_horizon": 5}, "Long": {"ma": 10, "ret": 10, "vol": 10, "roc_thresh": 0.02, "slope_horizon": 10}, "Daily": {"ma": 7, "ret": 1, "vol": 5, "roc_thresh": 0.02, "slope_horizon": 1}}, "candle_params": {"doji_base": 0.01, "doji_scale": 0.02, "doji_min": 0.01, "doji_max": 0.1, "long_body_base": 0.3, "long_body_scale": 0.3, "long_body_min": 0.3, "long_body_max": 0.6, "small_body_base": 0.15, "small_body_scale": 0.1, "small_body_min": 0.15, "small_body_max": 0.25, "shadow_ratio_base": 1.2, "shadow_ratio_scale": 0.8, "shadow_ratio_min": 1.2, "shadow_ratio_max": 2.0, "near_edge": 0.25, "highvol_spike": 1.5, "lowvol_dip": 0.7, "hammer_base": 0.15, "hammer_scale": 0.1, "hammer_min": 0.15, "hamme

In [29]:
import json
mbsetting = load_settings("tier1")

config_json = json.dumps(mbsetting)
print(config_json)

{"timeframe_map": {"Daily": 1, "Short": 3}, "phases": {"phase1": {"topN": 75}, "phase2": {"topN": 25}, "phase3": {"topN": 5}}, "profiles": {"Short": {"ma": 2, "ret": 1, "vol": 3, "roc_thresh": 0.02, "slope_horizon": 1}, "Swing": {"ma": 5, "ret": 5, "vol": 5, "roc_thresh": 0.02, "slope_horizon": 5}, "Long": {"ma": 10, "ret": 10, "vol": 10, "roc_thresh": 0.02, "slope_horizon": 10}, "Daily": {"ma": 7, "ret": 1, "vol": 5, "roc_thresh": 0.02, "slope_horizon": 1}}, "candle_params": {"doji_base": 0.025, "doji_scale": 0.02, "doji_min": 0.01, "doji_max": 0.09, "long_body_base": 0.4, "long_body_scale": 0.3, "long_body_min": 0.3, "long_body_max": 0.6, "small_body_base": 0.15, "small_body_scale": 0.1, "small_body_min": 0.15, "small_body_max": 0.25, "shadow_ratio_base": 1.2, "shadow_ratio_scale": 0.8, "shadow_ratio_min": 1.2, "shadow_ratio_max": 2.0, "near_edge": 0.25, "highvol_spike": 1.5, "lowvol_dip": 0.7, "hammer_base": 0.15, "hammer_scale": 0.1, "hammer_min": 0.15, "hammer_max": 0.25, "maruboz

In [32]:
from delta.tables import DeltaTable

# Option 1: Read directly via Spark table name
sdf = spark.table("bsf.companystockhistory_watermark")

# Option 2: Read via DeltaTable path
# delta_path = "/srv/lakehouse/tables/bsf.db/companystockhistory_watermark"
# sdf = DeltaTable.forPath(spark, delta_path).toDF()

# Show the latest rows
sdf.orderBy("CompanyId", "LastLoadedDate", ascending=False).show(10)


+---------+--------------+
|CompanyId|LastLoadedDate|
+---------+--------------+
|   322964|    2025-10-02|
|   322855|    2025-10-02|
|   322797|    2025-10-02|
|   322753|    2025-10-02|
|   322622|    2025-10-02|
|   322587|    2025-10-02|
|   321969|    2025-10-02|
|   321957|    2025-10-02|
|   321955|    2025-10-02|
|   321954|    2025-10-02|
+---------+--------------+
only showing top 10 rows



In [36]:
wm_df = spark.sql(f"""
    SELECT *
    FROM bsf.companystockhistory_watermark
    WHERE CompanyId = 52
""").toPandas()
wm_dict = dict(zip(wm_df.CompanyId, wm_df.LastLoadedDate))
print(wm_df)

   CompanyId LastLoadedDate
0         52     2025-10-01


In [34]:
print(wm_df)

   CompanyId LastLoadedDate
0         52     2025-10-02


In [35]:
from delta.tables import DeltaTable
from pyspark.sql import functions as F

# Reference the table
table_name = "bsf.companystockhistory_watermark"
dt = DeltaTable.forName(spark, table_name)

# Update CompanyId = 52 to new date
dt.update(
    condition = F.col("CompanyId") == 52,
    set = { "LastLoadedDate": F.lit("2025-10-01") }
)


In [11]:
from pyspark.sql import functions as F

sdf = spark.table("bsf.companystockhistory_watermark")

# Count rows per timeframe
sdf.groupBy("CompanyId") \
   .agg(F.max("LastLoadedDate").alias("max_date")) \
   .orderBy("CompanyId") \
   .show(truncate=False)

        watermark_update_df = sdf.groupBy("CompanyId") \
                               .agg(F.max("StockDate").alias("LastLoadedDate"))

+---------+----------+
|CompanyId|max_date  |
+---------+----------+
|52       |2025-10-02|
|91       |2025-10-02|
|138      |2025-10-02|
|145      |2025-10-02|
|151      |2025-10-02|
|205      |2025-10-02|
|216      |2025-10-02|
|253      |2025-10-02|
|320      |2025-10-02|
|325      |2025-10-02|
|332      |2025-10-02|
|383      |2025-10-02|
|397      |2025-10-02|
|424      |2025-10-02|
|437      |2025-10-02|
|458      |2025-10-02|
|471      |2025-10-02|
|494      |2025-10-02|
|497      |2025-10-02|
|521      |2025-10-02|
+---------+----------+
only showing top 20 rows



In [ ]:
from delta.tables import DeltaTable
dt = DeltaTable.forName(spark, "bsf.history_signals")
dt.vacuum(retentionHours=0)  # ⚠ deletes unreferenced files immediately


In [ ]:
from delta.tables import DeltaTable
dt = DeltaTable.forName(spark, "bsf.history_signals")
dt.history().show(truncate=False)



In [ ]:
spark.read.format("delta").load("/srv/lakehouse/tables/bsf.db/history_signals") \
     .select("TimeFrame").distinct().show()



In [ ]:
count = spark.sql(f"""
    SELECT COUNT(DISTINCT CompanyId) as cnt
    FROM bsf.company
    WHERE ListingExchange IN (1,2,3,16)
      AND Active = 1
      AND LastClose < 0.1
      AND LastHistoryDate >= date_sub(current_date(), 30)
""").collect()[0]["cnt"]

print(count)




In [ ]:
sdf = spark.table("bsf.companystockhistory")
sdf.toPandas().describe()

In [ ]:
sdf = spark.table("bsf.companystockhistory")
counts_df = (
    sdf.groupBy("CompanyId")
      .agg(F.count("*").alias("row_count"))
      .orderBy("CompanyId")
)

counts_df.show(50, truncate=False)
#pdf.head(10).T

In [ ]:
sdf = spark.table("bsf.companystockhistory_watermark")
pdf = sdf.limit(10).toPandas()
#pdf.head(10).T

In [ ]:
sdf = spark.table("bsf.history_signals_last")
pdf = sdf.limit(10).toPandas()
pdf

In [ ]:
sdf = spark.table("bsf.signaldriver")
sdf.toPandas().describe()



In [ ]:
sdf = spark.table("bsf.history_signals")
pdf = sdf.limit(10).toPandas()
pdf

In [ ]:
sdf = spark.table("bsf.final_candidates")
pdf = sdf.limit(10).toPandas()
pdf

In [ ]:
sdf = spark.table("bsf.company")
# Group by CompanyId, TimeFrame, StockDate
counts_df = (
    sdf.groupBy("ListingExchange")
      .agg(F.count("*").alias("row_count"))
      .orderBy("ListingExchange")
)

counts_df.show(50, truncate=False)



sdf = spark.table("bsf.history_signals")

# Group by CompanyId, TimeFrame, StockDate
counts_df = (
    sdf.groupBy( "TimeFrame")
      .agg(F.count("*").alias("row_count"))
      .orderBy( "TimeFrame")
)

counts_df.show(50, truncate=False)


In [ ]:
# Assume you have a Spark DataFrame
sdf = spark.table("bsf.companystockhistory")

# 1️⃣ Show the schema (data types included)
sdf.printSchema()

# 2️⃣ Get a list of column names
# print(sdf.columns)

# 3️⃣ Show first few rows with all columns (default shows truncated view)
sdf.show(truncate=False)  

# 4️⃣ For more detailed metadata about columns
sdf.dtypes  # Returns a list of (column_name, data_type)


In [ ]:
# 🚨 Safety stop — prevents accidental full execution
raise RuntimeError("⚠️ This notebook is blocked. Do NOT run all cells without checking!")
